**Network Strength Testing**

Notebook that allows testing of LCZero against itself for different IDs, made by enelenes with patrick's guidance.

All cells before the one with the form, should be run once, then you can fiddle with parameters in the form, and rerun the following cell.

In [1]:
import subprocess
if subprocess.run(["nvidia-smi", "-L"]).returncode != 0:
  assert False, "No GPU available!"

!apt-get update
!apt-get install -y --fix-missing --no-install-recommends cuda-compiler-9-2 cuda-cublas-dev-9-2 cuda-cudart-dev-9-2

!wget http://developer.download.nvidia.com/compute/redist/cudnn/v7.3.1/cudnn-9.2-linux-x64-v7.3.1.20.tgz
!cd /usr/local && tar -xzvf /content/cudnn-9.2-linux-x64-v7.3.1.20.tgz
!chmod a+r /usr/local/cuda/lib64/libcudnn*

!apt-get install ninja-build libprotobuf-dev protobuf-compiler
!pip3 install meson

!rm -rf lc0
!git clone --recurse-submodules https://github.com/LeelaChessZero/lc0.git
!cd lc0 && git checkout $(git tag --list |grep -v rc |tail -1)

# Build Lc0
!cd lc0 && rm -rf build
!cd lc0 && meson build --buildtype release -Db_lto=true -Dgtest=false
!cd lc0/build && ninja

# Create dir for network
!mkdir -p networks/

# Download network 32684 (or other network that you need) in networks dir
# Change dir
%cd networks
# Download it, get the link from Lc0 site
!wget http://lczero.org/get_network?sha=692a3a539956498fa3ca407f7faa39e70240de91d79b261a4128e392dc7b86f1


# Check the contents of networks dir to rename the long filename, check the correct long filename and use it in Step 100
!ls -la

# Step 100, Rename network long filename to w32684, format !mv <long filename> <short filename>
!mv get_network?sha=692a3a539956498fa3ca407f7faa39e70240de91d79b261a4128e392dc7b86f1 w32684

# Download network 32600
!wget http://lczero.org/get_network?sha=274663ba5d178e25061c353c8fe15fd8f4b6cd2f6bc15820fe7900f269fa623f

#Rename network long filename to w32600, similar to Step 100.
!mv get_network?sha=274663ba5d178e25061c353c8fe15fd8f4b6cd2f6bc15820fe7900f269fa623f w32600


# Check that you have the weights, w32684 and w32600
!ls -la

# Go back to previous dir (command is %cd double dots), important!!
%cd ..

# Check your current dir name, it should be /content/ or /content
!pwd


# Download start pgn and cutechess
!wget https://cdn.discordapp.com/attachments/433622350626422795/435942405913575435/2moves_LT_1000.pgn # 2 move opening database
!wget https://github.com/cutechess/cutechess/releases/download/gui-1.0.0/cutechess-cli-1.0.0-linux64.tar.gz
!tar -xvf cutechess-cli-1.0.0-linux64.tar.gz

# Run a match, adjust engine name, weights rounds tc and others.
!cd cutechess-cli && QT_LOGGING_RULES="*.debug=true" ./cutechess-cli -engine name="Lc0 v0.20.1 32684" cmd=/content/lc0/build/lc0 arg="--weights=/content/networks/w32684" stderr=leela1.err -engine name="Lc0 v0.20.1 32600" cmd=/content/lc0/build/lc0 arg="--weights=/content/networks/w32600" stderr=leela2.err -each tc=0/10+0.1 proto=uci -recover -games 2 -rounds 1 -repeat -pgnout output_pgn.pgn -openings file=../2moves_LT_1000.pgn order=random


FileNotFoundError: [WinError 2] The system cannot find the file specified

Download PGN files with played games 

In [2]:
# Download the match games
from google.colab import files
files.download('cutechess-cli/output_pgn.pgn')

ModuleNotFoundError: No module named 'google.colab'